## Part a) Box Picking Example in BotOp

In [2]:
from robotic import ry
import numpy as np
import time
ry.params_add({'physx/motorKp': 10000., 'physx/motorKd': 1000., 'physx/angularDamping': 10., 'physx/defaultFriction': 100.})
ry.params_print()

-- ry.cpp:operator():86(0) python,
physx/motorKp: 10000,
physx/motorKd: 1000,
physx/angularDamping: 10,
physx/defaultFriction: 100


In [3]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))
C.view(False)

0

In [4]:
#   .setShape(ry.ST.sphere, size=[.025]) \
C.addFrame('box') \
    .setPosition([-.25,.1,.675]) \
    .setShape(ry.ST.ssBox, size=[.05,.05,.05,.005]) \
    .setColor([1,.5,0]) \
    .setMass(.1) \
    .setContact(True)
C.view()

0

In [5]:
# WAYPOINT ENGINEERING:
# manually define frames as an endeff waypoints, relative to box:
way0 = C.addFrame('way0', 'box')
way1 = C.addFrame('way1', 'box')

In [6]:
way0.setShape(ry.ST.marker, size=[.1])
way0.setRelativePose('t(0 0 .1) d(90 0 0 1)')

way1.setShape(ry.ST.marker, size=[.1])
way1.setRelativePose('d(90 0 0 1)')

C.view()

In [8]:
# define a 2 waypoint problem in KOMO
komo = ry.KOMO()
komo.setConfig(C, True)
komo.setTiming(2., 1, 5., 0)
komo.addControlObjective([], 0, 1e-0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq);
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq);
komo.addObjective([1.], ry.FS.poseDiff, ['l_gripper', 'way0'], ry.OT.eq, [1e1]);
komo.addObjective([2.], ry.FS.poseDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1]);

In [9]:
ret = ry.NLP_Solver() \
    .setProblem(komo.nlp()) \
    .setOptions( stopTolerance=1e-2, verbose=4 ) \
    .solve()
print(ret)

{ time: 0.00222, evals: 18, done: 1, feasible: 1, sos: 4.37426, f: 0, ineq: 0, eq: 0.000146417 }
====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
----newton---- initial point f(x):93.1593 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):    69.6169  ACCEPT
--newton-- it:   2  |Delta|:        0.2  alpha:          1  evals:   3  f(y):    47.7566  ACCEPT
--newton-- it:   3  |Delta|:        0.2  alpha:          1  evals:   4  f(y):    28.1096  ACCEPT
--newton-- it:   4  |Delta|:        0.2  alpha:          1  evals:   5  f(y):    11.4588  ACCEPT
--newton-- it:   5  |Delta|:        0.2  alpha:          1  evals:   6  f(y):    6.02281  ACCEPT
--newton-- it:   6  |Delta|:        0.2  alpha:          1  evals:   7  f(y):     4.0943  ACCEPT
--newton-- it:   7  |Delta|:   0.170104  alpha:          1  evals:   8  f(y):    3.57494  ACCEPT
--newton-- it:   8  |Delta|:  0.0579509  alpha:          1  evals:   9  f(y

In [10]:
komo.view(False, "waypoints solution")

0

In [11]:
komo.view_close()
path = komo.getPath()

In [12]:
bot = ry.BotOp(C, False)
bot.home(C)

In [20]:
bot.gripperMove(ry._left)
#bot.gripperOpen(ry._left)
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

In [21]:
bot.move(path, [2., 3.])
while bot.getTimeToEnd()>0:
    bot.sync(C, .1)

In [22]:
bot.gripperCloseGrasp(ry._left, 'box')
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

In [23]:
bot.home(C)

In [24]:
bot.gripperMove(ry._left)
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

In [26]:
del bot
del C

NameError: name 'bot' is not defined

## Part b) Box Pushing Example

In [1]:
from robotic import ry
import numpy as np
import time

print('ry version', ry.__version__, ry.compiled())
ry.params_add({'physx/motorKp': 10000., 'physx/motorKd': 1000.})
ry.params_print()

ry version 0.0.22 compile time: Oct  8 2023 16:07:43
-- ry.cpp:operator():86(0) python,
physx/motorKp: 10000,
physx/motorKd: 1000


In [2]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))
C.view(False)

0

In [3]:
C.addFrame('box') \
    .setPosition([-.25,.1,0.725]) \
    .setShape(ry.ST.ssBox, size=[.15,.15,.15,.005]) \
    .setColor([1,.5,0]) \
    .setMass(.1) \
    .setContact(True)
C.view()

0

In [4]:
# WAYPOINT ENGINEERING:
# manually define frames as an endeff waypoints, relative to box:
way0 = C.addFrame('way0', 'box')
way1 = C.addFrame('way1', 'box')

way0.setShape(ry.ST.marker, size=[.1])
way0.setRelativePose('t(-.07 0 0) d(90 0 0 1)')

way1.setShape(ry.ST.marker, size=[.1])
way1.setRelativePose('t(.2 0 0) d(90 0 0 1)')

C.view()

0

In [5]:
# define a 2 waypoint problem in KOMO
komo = ry.KOMO()
komo.setConfig(C, True)
komo.setTiming(2., 1, 5., 0)
komo.addControlObjective([], 0, 1e-0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq);
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq);
komo.addObjective([1.], ry.FS.poseDiff, ['l_gripper', 'way0'], ry.OT.eq, [1e1]);
komo.addObjective([2.], ry.FS.poseDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1]);

In [6]:
ret = ry.NLP_Solver() \
    .setProblem(komo.nlp()) \
    .setOptions( stopTolerance=1e-2, verbose=4 ) \
    .solve()
print(ret)

{ time: 0.002497, evals: 18, done: 1, feasible: 1, sos: 4.35847, f: 0, ineq: 0, eq: 0.104068 }
====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
----newton---- initial point f(x):90.5491 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):    65.6806  ACCEPT
--newton-- it:   2  |Delta|:        0.2  alpha:          1  evals:   3  f(y):    43.6545  ACCEPT
--newton-- it:   3  |Delta|:        0.2  alpha:          1  evals:   4  f(y):    25.2704  ACCEPT
--newton-- it:   4  |Delta|:        0.2  alpha:          1  evals:   5  f(y):    11.4206  ACCEPT
--newton-- it:   5  |Delta|:        0.2  alpha:          1  evals:   6  f(y):    5.64852  ACCEPT
--newton-- it:   6  |Delta|:        0.2  alpha:          1  evals:   7  f(y):    3.91218  ACCEPT
--newton-- it:   7  |Delta|:   0.124599  alpha:          1  evals:   8  f(y):    3.59406  ACCEPT
--newton-- it:   8  |Delta|:  0.0531028  alpha:          1  evals:   9  f(y):

In [7]:
komo.view(False, "waypoints solution")

0

In [8]:
komo.view_close()
path = komo.getPath()

In [9]:
bot = ry.BotOp(C, False)
bot.home(C)

In [10]:
bot.gripperClose(ry._left)
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

In [12]:
bot.moveTo(path[0])
bot.moveTo(path[1])
while bot.getTimeToEnd()>0:
    bot.sync(C, .1)

In [13]:
bot.home(C)

In [14]:
bot.gripperMove(ry._left)
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

In [15]:
del bot
del C

-- bot.cpp:~BotOp:112(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:56(0) shutting down SimThread
-- simulation.cpp:~Simulation:148(0) shutting down Simulation
